In [1]:
#import packages
import requests
import pandas as pd
import json
import collections
import psycopg2

In [2]:
#function used later on for data wrangling
def flatten(d, parent_key='', sep='_'):
    items = []
    for k, v in d.items():
        new_key = parent_key + sep + k if parent_key else k
        if isinstance(v, collections.MutableMapping):
            items.extend(flatten(v, new_key, sep=sep).items())
        else:
            items.append((new_key, v))
    return dict(items)

In [3]:
#db connection - would parameterize/ask to provide instead of hardcoding into the code with more time
conn = psycopg2.connect(
    host="chcubs.crtnht6h1zib.us-east-1.rds.amazonaws.com",
    database="xpyymgxv",
    user="xpyymgxv",
    password="cepaxbvn")

In [4]:
#assuming the job is running regularly without failure, select the latest game_date with data present already
#create date_to_run parameter based off the max game_date + 1 to pull information available
cursor = conn.cursor()
cursor.execute("select max(game_date) from public.game_data")
data=cursor.fetchall()
column_names=['game_Date']
df_from_db=pd.DataFrame(data, columns=column_names)
date_to_run=max(df_from_db['game_Date'])[5:7]+'/'+str.zfill(str(int(max(df_from_db['game_Date'])[8:])+1),2)+'/'+max(df_from_db['game_Date'])[:4]

In [5]:
#mocked up 2 rows of data, the 'latest' game insert was on 08/02/2018
date_to_run

'08/03/2018'

In [6]:
#pull information from schedule API as that has a lot of information needed
#this is assuming the job runs relatively late in the evening, most games of the day are finished
#will likely have to 'tune' what the best time to schedule the job is, as evening baseball games can run fairly late
#want to make sure no data is falling out from collection based on date selection for API call
schedule_response = requests.get("https://statsapi.mlb.com/api/v1/schedule/?sportId=1&scheduleTypes=games&date="+date_to_run+"&fields=dates,date,games,gamePK,link,gameDate,status,statusCode,teams,score,team,id,name,venue,id,name")
schedule=schedule_response.json()
df_schedule=pd.json_normalize(schedule,record_path=['dates','games'])

In [7]:
#filtering for statusCode='F' to capture completed games
df_schedule=df_schedule[df_schedule['status.statusCode']=='F'].copy()
#grabbed the gamePK from the link - with time would analyze more efficient way to grab gamePK
df_schedule['gamePK']=df_schedule['link'].apply(lambda x: x.split('/')[4])
#parsing out game date and game time from the gameDate field 
df_schedule['game_Date']=pd.to_datetime(df_schedule['gameDate']).dt.date
df_schedule['game_Time']=pd.to_datetime(df_schedule['gameDate']).dt.time

In [8]:
df_games=df_schedule[['gamePK','game_Date','game_Time','teams.away.team.id','teams.away.team.name',
                      'teams.home.team.id', 'teams.home.team.name','teams.away.score','teams.home.score',
                      'venue.id', 'venue.name']].copy()
df_games.rename(columns={'teams.away.team.id':'away_Team_ID','teams.away.team.name':'away_Team_Name',
                         'teams.home.team.id':'home_Team_ID','teams.home.team.name':'home_Team_Name',
                         'teams.away.score':'away_Team_Score','teams.home.score':'home_Team_Score',
                         'venue.id':'venue_ID', 'venue.name':'venue_Name'},inplace=True)

In [9]:
#high level game level information is gathered, need to add more on game level information though
df_games.head()

,gamePK,game_Date,game_Time,away_Team_ID,away_Team_Name,home_Team_ID,home_Team_Name,away_Team_Score,home_Team_Score,venue_ID,venue_Name
0,531060,2018-08-03,18:20:00,135,San Diego Padres,112,Chicago Cubs,4.0,5.0,17,Wrigley Field
1,531062,2018-08-03,23:05:00,138,St. Louis Cardinals,134,Pittsburgh Pirates,6.0,7.0,31,PNC Park
2,531058,2018-08-03,23:05:00,146,Miami Marlins,143,Philadelphia Phillies,1.0,5.0,2681,Citizens Bank Park
3,531057,2018-08-03,23:10:00,108,Los Angeles Angels,114,Cleveland Indians,7.0,4.0,5,Progressive Field
4,531049,2018-08-03,23:10:00,144,Atlanta Braves,121,New York Mets,2.0,1.0,3289,Citi Field


In [10]:
#going to boxscore API to get total number of pitches made per team per game
#following API calls are made on a loop per completed game per day
games=df_games['gamePK'].unique()
daily_away_pitches=[]
daily_home_pitches=[]
for g in range(len(games)):
    pitches_response = requests.get("https://statsapi.mlb.com/api/v1/game/"+str(games[g])+"/boxscore?fields=teams,team,players,person,id,fullName,stats,pitching,pitchesThrown,allPositions,code,name")
    pitches=pitches_response.json()
    #flatten the data from the API
    #grab the IDs of pitchers
    #sum the number of pitches made per pitcher per team 
    pitches_flatten=flatten(pitches)
    away_game_pitches=0
    home_game_pitches=0
    for i in range(len(pitches_flatten['teams_away_pitchers'])):
        away_game_pitches += pitches_flatten['teams_away_players_ID'+str(pitches_flatten['teams_away_pitchers'][i])+'_stats_pitching_pitchesThrown']
    daily_away_pitches.append(away_game_pitches)
    for h in range(len(pitches_flatten['teams_home_pitchers'])):
        home_game_pitches += pitches_flatten['teams_home_players_ID'+str(pitches_flatten['teams_home_pitchers'][h])+'_stats_pitching_pitchesThrown']
    daily_home_pitches.append(home_game_pitches)

<ipython-input-2-fe36af0cd986>:5: DeprecationWarning: Using or importing the ABCs from 'collections' instead of from 'collections.abc' is deprecated since Python 3.3, and in 3.9 it will stop working
  if isinstance(v, collections.MutableMapping):


In [11]:
#create dataframe of gamePK, away team pitches, home team pitches
df_pitches=pd.DataFrame({'gamePK':games,'away_Team_Pitches':daily_away_pitches, 'home_Team_Pitches':daily_home_pitches})

In [12]:
#merge with original dataframe as more information about a game is brought in
df_games=df_games.merge(df_pitches,on='gamePK')

In [13]:
df_games.head()

,gamePK,game_Date,game_Time,away_Team_ID,away_Team_Name,home_Team_ID,home_Team_Name,away_Team_Score,home_Team_Score,venue_ID,venue_Name,away_Team_Pitches,home_Team_Pitches
0,531060,2018-08-03,18:20:00,135,San Diego Padres,112,Chicago Cubs,4.0,5.0,17,Wrigley Field,133,158
1,531062,2018-08-03,23:05:00,138,St. Louis Cardinals,134,Pittsburgh Pirates,6.0,7.0,31,PNC Park,135,187
2,531058,2018-08-03,23:05:00,146,Miami Marlins,143,Philadelphia Phillies,1.0,5.0,2681,Citizens Bank Park,156,138
3,531057,2018-08-03,23:10:00,108,Los Angeles Angels,114,Cleveland Indians,7.0,4.0,5,Progressive Field,144,147
4,531049,2018-08-03,23:10:00,144,Atlanta Braves,121,New York Mets,2.0,1.0,3289,Citi Field,138,117


In [14]:
#going to play by play API to get max launchSpeed and batter with max launchSpeed per game
#assuming this data is needed at a game level
#assuming the end user only cares about the batter with the max launchSpeed across away and home games given how instructions were framed
#if needed by team, can be broken out 
#might consider home_team_max_Speed, home_team_max_Speed_Batter, away_team_max_Speed, away_team_max_Batter
#also might consider adding this information into the player table, depending on end user needs
max_Speeds=[]
max_Speeds_Batters=[]
for g in range(len(games)):
    speed_response = requests.get("https://statsapi.mlb.com/api/v1/game/"+str(games[g])+"/playByPlay?fields=allPlays,matchup,batter,id,fullName,playEvents,details,hitData,launchSpeed")
    speed=speed_response.json()
    df_speed=pd.json_normalize(speed,record_path=['allPlays'])
    x=[]
    #given time restraints, the one game I looked at had speed in the PlayEvent so I dropped all earlier onces
    #would do analysis before productionalizing code
    for y in range(len(df_speed['playEvents'])):
        x.append(df_speed['playEvents'][y][len(df_speed['playEvents'][y])-1])
    df_speed['speed']=x
    l1=[]
    for a in range(len(df_speed['speed'])):
        if ('hitData' in df_speed['speed'][a].keys()) and ('launchSpeed' in df_speed['speed'][a]['hitData'].keys()):
            s=df_speed['speed'][a]['hitData']['launchSpeed']
        else:
            s=0.0
        l1.append(s)
    df_speed['max_Speed']=l1
    max_Speeds.append(max(df_speed['max_Speed']))
    max_Speeds_Batters.append(df_speed[df_speed['max_Speed']==(max(df_speed['max_Speed']))]['matchup.batter.fullName'].values[0])

In [15]:
df_bat_speeds=pd.DataFrame({'gamePK':games,'highest_Exit_Velocity':max_Speeds, 'Batter':max_Speeds_Batters})
df_games=df_games.merge(df_bat_speeds,on='gamePK')

In [16]:
df_games.head()

,gamePK,game_Date,game_Time,away_Team_ID,away_Team_Name,home_Team_ID,home_Team_Name,away_Team_Score,home_Team_Score,venue_ID,venue_Name,away_Team_Pitches,home_Team_Pitches,highest_Exit_Velocity,Batter
0,531060,2018-08-03,18:20:00,135,San Diego Padres,112,Chicago Cubs,4.0,5.0,17,Wrigley Field,133,158,109.7,Hunter Renfroe
1,531062,2018-08-03,23:05:00,138,St. Louis Cardinals,134,Pittsburgh Pirates,6.0,7.0,31,PNC Park,135,187,111.1,Marcell Ozuna
2,531058,2018-08-03,23:05:00,146,Miami Marlins,143,Philadelphia Phillies,1.0,5.0,2681,Citizens Bank Park,156,138,106.5,Brian Anderson
3,531057,2018-08-03,23:10:00,108,Los Angeles Angels,114,Cleveland Indians,7.0,4.0,5,Progressive Field,144,147,110.7,Shohei Ohtani
4,531049,2018-08-03,23:10:00,144,Atlanta Braves,121,New York Mets,2.0,1.0,3289,Citi Field,138,117,103.4,Kurt Suzuki


In [17]:
#I pute Team Info into a separate table because it felt to be at a different level
#all earlier data points were at the game level
#Team Info (ID & names) are arguably at the player per game level
#This can change as people get injured, etc so I captured this data at a player per game level
players_response = requests.get("https://statsapi.mlb.com/api/v1/game/531060/boxscore?fields=teams,team,players,person,id,fullName")
players=players_response.json()
players_flatten=flatten(players)

In [18]:
home_player_names=[]
away_player_names=[]
home_player_ids=[]
away_player_ids=[]
home_team_ids=[]
away_team_ids=[]
home_gamePKs=[]
away_gamePKs=[]
#max_players=[]
for g in range(len(games)):
    players_response = requests.get("https://statsapi.mlb.com/api/v1/game/"+games[g]+"/boxscore?fields=teams,team,players,person,id,fullName")
    players=players_response.json()
    players_flatten=flatten(players)
    home_players=list(set(players_flatten['teams_home_batters']+players_flatten['teams_home_pitchers']+players_flatten['teams_home_bench']+players_flatten['teams_home_bullpen']))
    away_players=list(set(players_flatten['teams_away_batters']+players_flatten['teams_away_pitchers']+players_flatten['teams_away_bench']+players_flatten['teams_away_bullpen']))
    for n in range(len(home_players)):
        home_player_names.append(players_flatten['teams_home_players_ID'+str(home_players[n])+'_person_fullName'])
    for n in range(len(away_players)):
        away_player_names.append(players_flatten['teams_away_players_ID'+str(away_players[n])+'_person_fullName'])
    home_player_ids.extend(home_players)
    away_player_ids.extend(away_players)
    home_team_ids.extend([players_flatten['teams_home_team_id']] * len(home_players))
    away_team_ids.extend([players_flatten['teams_away_team_id']] * len(away_players))
    home_gamePKs.extend([games[g]] * len(home_players))
    away_gamePKs.extend([games[g]] * len(away_players))

In [19]:
game_IDs=home_gamePKs + away_gamePKs
team_IDs=home_team_ids +away_team_ids
player_IDs=home_player_ids + away_player_ids
player_names=home_player_names + away_player_names

In [20]:
df_players_per_game=pd.DataFrame({'gamePK':pd.Series(game_IDs),'team_ID':pd.Series(team_IDs), 'player_ID':pd.Series(player_IDs), 'player_Name':pd.Series(player_names)})

In [21]:
df_players_per_game.head()

,gamePK,team_ID,player_ID,player_Name
0,531060,112,600968,Randy Rosario
1,531060,112,518792,Jason Heyward
2,531060,112,450314,Ben Zobrist
3,531060,112,518553,Steve Cishek
4,531060,112,445213,Brandon Kintzler


In [22]:
df_games.head()

,gamePK,game_Date,game_Time,away_Team_ID,away_Team_Name,home_Team_ID,home_Team_Name,away_Team_Score,home_Team_Score,venue_ID,venue_Name,away_Team_Pitches,home_Team_Pitches,highest_Exit_Velocity,Batter
0,531060,2018-08-03,18:20:00,135,San Diego Padres,112,Chicago Cubs,4.0,5.0,17,Wrigley Field,133,158,109.7,Hunter Renfroe
1,531062,2018-08-03,23:05:00,138,St. Louis Cardinals,134,Pittsburgh Pirates,6.0,7.0,31,PNC Park,135,187,111.1,Marcell Ozuna
2,531058,2018-08-03,23:05:00,146,Miami Marlins,143,Philadelphia Phillies,1.0,5.0,2681,Citizens Bank Park,156,138,106.5,Brian Anderson
3,531057,2018-08-03,23:10:00,108,Los Angeles Angels,114,Cleveland Indians,7.0,4.0,5,Progressive Field,144,147,110.7,Shohei Ohtani
4,531049,2018-08-03,23:10:00,144,Atlanta Braves,121,New York Mets,2.0,1.0,3289,Citi Field,138,117,103.4,Kurt Suzuki


In [ ]:
#given time constraints I did not include the upsert logic
#for the two tables I created for this exercise, I would do an upsert by gamePK
#if the gamePK already exists, I would delete the older records and replace them with the new,
#both if the job runs incrementally and if the job were to run for a specific gamePK
#given time constraints I also did not create a specific function for how I would restate a single game
#The player table is easy to restate as that data is pulled by specifying the gamePK in the API call
#The general df_games table would require a lookup of finding 
#the date that corresponds to the gamePK that needs to be restated.
#I would then also add a line to filter the fine df_games created to only include a specific PK before doing the upsert:
#for example:
#df_games=df_games[df_games['gamePK']=XXXXXX].copy()